In [1]:
# %load ../ssq_dream.py

import numpy as np
import pandas as pd
import random
from glob import glob
from tqdm import tqdm
from itertools import combinations
from nltk.util import ngrams

df = pd.read_csv("ssq.csv")
print(df.tail().loc[:, "id":"blue"])

           id        date  red1  red2  red3  red4  red5  red6  blue
2507  2019149  2019-12-26     1     6    27    28    31    33     7
2508  2019150  2019-12-29     2     9    14    22    27    29     2
2509  2019151  2019-12-31     2     6     9    18    24    26    14
2510  2020001  2020-01-02     2    15    23    26    29    30     2
2511  2020002  2020-01-05     4     9    14    15    16    29    11


In [5]:
# 确定red_tag

# ![](http://img5.cache.netease.com/help/2013/2/5/20130205200221990eb.png)

bonus = {
    (0, 1): 5,
    (1, 1): 5,
    (2, 1): 5,
    (3, 1): 10,
    (4, 0): 10,
    (4, 1): 200,
    (5, 0): 200,
    (5, 1): 3000,
    (6, 0): 5000000,
    (6, 1): 10000000,
}

# 检查列表

topic, n = "ssq", 6

hot_water = sorted(glob(f"gamble/{topic}/*-{topic}.p"))[-1]
date = hot_water[-16:-6]
df_dream = pd.read_csv(hot_water)

last_shit = df.loc[df.date <= date, "date"].max()
df_last = df.loc[df.date == last_shit, df_dream.columns]
list_last = df_last.values[0].tolist()


def getd(v):
    return len(set(v[:n]) & set(list_last[:n])), len(set(v[n:]) & set(list_last[n:]))


df_dream["check"] = df_dream.apply(getd, axis=1)
df_dream["bonus"] = df_dream.check.apply(lambda _: bonus.get(_, 0))
print(df_dream)
print("bonus", df_dream["bonus"].sum())

      red1  red2  red3  red4  red5  red6  blue   check  bonus
1045     1     7    10    14    21    25    12  (1, 0)      0
30      11    17    20    22    28    32     1  (0, 0)      0
85       5    12    16    18    26    30    10  (1, 0)      0
50       4    13    15    17    24    27    14  (2, 0)      0
143      6     8    19    25    29    32     7  (1, 0)      0
bonus 0


In [3]:
blue = df.blue.iloc[-1]
td = pd.datetime.today().date()
year = str(td.year)
yall = df.date.str.slice(0, 4)
ynum = yall.nunique() - 1
idx = yall == year

# other years blue average
key = pd.Series(ngrams(df.loc[~idx, "blue"], 2)).value_counts().to_frame("key")
dfkey = key.loc[[_ for _ in key.index if _[0] == blue]] / ynum
# this year blue
key2 = pd.Series(ngrams(df.loc[idx, "blue"], 2)).value_counts().to_frame("key2")
dfkey2 = key2.loc[[_ for _ in key2.index if _[0] == blue]]

dfkey = dfkey.join(dfkey2, how="left").fillna(0)
dfkey["key3"] = dfkey["key2"] - dfkey["key"]
dfkey.sort_values(by="key3", inplace=True)
blued = [b for _, b in dfkey.head().index]

In [12]:
key['bluea'] = [_[0] for _ in key.index]
key['blueb'] = [_[1] for _ in key.index]

In [4]:
# # 输出
columns = ["red1", "red2", "red3", "red4", "red5", "red6", "blue"]
idxa = (df.red1 <= 11) & (df.red6 >= 18)
t = len(df)
r = len(df[idxa]) / t * 100
bset = df.head(-1).loc[idxa, "red1":"red6"].apply(set, axis=1)
tail = set(df.iloc[-1]["red1":"red6"])

# 剔除与最后一期交集>3
star = bset.apply(lambda _: len(_ & tail))
bar = bset.loc[star[star < 3].index]
ia = np.random.choice(bar.index)
idx, obj = [ia], bar[ia]
for i in range(4):
    ib = np.random.choice(bar.apply(lambda _: len(_ & obj)).nsmallest(10).index)
    idx.append(ib)
    obj |= bar[ib]

ds = df.loc[idx, columns]
obj = set(np.reshape(ds.loc[:, "red1":"red6"].values, (1, 30))[0])
dr = sorted(set(range(1, 34)) - obj)
print(f"God bless! total {t} red1<=9 and red6>=23: {len(df[idxa])} {r:.1f}%")
print(f"ETL {len(bar)} unique {len(obj)} drop {dr}\n")

ds.blue = blued
ds.to_csv(f"gamble/ssq/{td}-ssq.p")
print(ds)

God bless! total 2512 red1<=9 and red6>=23: 2326 92.6%
ETL 2199 unique 27 drop [2, 3, 9, 23, 31, 33]

      red1  red2  red3  red4  red5  red6  blue
1045     1     7    10    14    21    25    12
30      11    17    20    22    28    32     1
85       5    12    16    18    26    30    10
50       4    13    15    17    24    27    14
143      6     8    19    25    29    32     7


# 写入数据库app.db

In [15]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///app.db', echo=False)

In [23]:
ds.assign(date=td, bonous=0).to_sql('lotteries', con=engine, if_exists='replace', index_label='id')

![](http://img5.cache.netease.com/help/2013/2/5/20130205200221990eb.png)

In [6]:
# 最大遗漏
b = df[["blue"]].reset_index()

b['z'] = b.groupby("blue").index.diff()

In [7]:
b.tail()

,index,blue,z
2507,2507,7,16.0
2508,2508,2,28.0
2509,2509,14,5.0
2510,2510,2,2.0
2511,2511,11,14.0


In [21]:
b.groupby("blue").z.describe(percentiles=np.arange(0,1,0.05))

,count,mean,std,min,0%,5%,10%,15%,20%,25%,...,55%,60%,65%,70%,75%,80%,85%,90%,95%,max
blue,,,,,,,,,,,,,,,,,,,,,
1,162.0,15.197531,16.067729,1.0,1.0,1.0,2.0,3.0,4.0,4.25,...,11.55,13.0,14.00,16.0,19.00,22.8,29.00,36.0,47.85,84.0
2,145.0,17.013793,16.986509,1.0,1.0,1.0,2.0,2.6,3.0,4.00,...,13.00,14.4,18.00,21.0,24.00,29.2,35.00,41.2,54.20,82.0
3,149.0,16.644295,16.456771,1.0,1.0,1.0,2.0,3.0,3.0,5.00,...,13.00,15.0,17.20,20.0,22.00,27.0,31.80,39.2,50.20,78.0
4,143.0,17.181818,18.705515,1.0,1.0,1.1,2.0,3.0,4.0,4.00,...,13.00,15.2,18.30,20.0,23.00,26.6,31.00,39.0,50.00,113.0
5,151.0,15.688742,13.073222,1.0,1.0,1.0,2.0,2.5,3.0,6.00,...,14.50,16.0,18.50,21.0,22.00,24.0,28.50,32.0,42.50,65.0
6,151.0,16.397351,15.838384,1.0,1.0,1.0,2.0,2.5,4.0,5.00,...,13.00,15.0,17.50,19.0,23.00,27.0,29.00,33.0,49.50,90.0
7,157.0,15.738854,17.205078,1.0,1.0,1.0,2.0,2.0,3.0,4.00,...,11.00,12.0,14.00,17.0,23.00,26.0,30.60,40.4,49.60,91.0
8,132.0,18.643939,18.757241,1.0,1.0,1.0,2.0,3.0,4.2,6.00,...,13.05,17.0,19.00,21.0,23.00,29.8,36.00,45.0,56.90,95.0
9,166.0,14.746988,12.985627,1.0,1.0,1.0,2.0,3.0,4.0,5.00,...,12.00,13.0,18.00,19.0,20.75,25.0,27.50,31.0,36.00,95.0


In [32]:
y=b.query('blue==16 and z>46')

In [40]:
y=b.query('z>70')
y

,index,blue,z
232,232,4,77.0
234,234,13,72.0
244,244,9,95.0
318,318,6,90.0
391,391,4,72.0
474,474,15,84.0
775,775,2,82.0
897,897,14,71.0
949,949,12,85.0
953,953,7,78.0


In [34]:
df.loc[y.index-1]

,id,date,red1,red2,red3,red4,red5,red6,blue,red1a,...,No2,No2_M,No3,No3_M,No4,No4_M,No5,No5_M,No6,No6_M
822,2008152,2008-12-25,1,4,6,22,26,30,8,30,...,46,595105,721,3000,34570,200,641698,10,7210856,5
1057,2010079,2010-07-11,8,11,12,14,18,22,2,18,...,112,209552,1042,3000,56869,200,1095352,10,5719876,5
1480,2013042,2013-04-14,1,8,12,13,15,33,3,15,...,131,279299,973,3000,54934,200,1142444,10,9424624,5
